# Graphs, tf.Example and tf.Tensor

Experiment with exploring graphs for tensors and combining tensors into example_pb2 strings

In [1]:
import tensorflow as tf

import os
import shutil
import datetime as dt

### Set up logs dir

In [2]:
LOGS_DIR_BASE = os.path.join(os.path.abspath('.'), 'get_logs')
shutil.rmtree(LOGS_DIR_BASE, ignore_errors=True)
os.mkdir(LOGS_DIR_BASE)

and sumary writer

In [3]:
def timestamp_str(): return dt.datetime.now().strftime("%Y_%m_%dT%H_%M_%S")

In [4]:
LOGDIR_PATH = os.path.join(LOGS_DIR_BASE, f'simple_examples')
SUMMARY_WRITER = tf.summary.create_file_writer(LOGDIR_PATH)

## Scalar tensors & math

Define tensor scalars

In [5]:
A = tf.constant(3)
B = tf.constant(4)

Will need a `tf.function` to trace graphs

In [17]:
@tf.function
def combine_two_tensors(first_tensor: tf.Tensor, second_tensor: tf.Tensor):
    return first_tensor+second_tensor


Write summary of the op that can be viewed using tensorboard

In [6]:
tf.summary.trace_on(graph=True, profiler=True)
c = combine_two_tensors(A, B)
with SUMMARY_WRITER.as_default():
    tf.summary.trace_export(
        name=f'simple_example_{timestamp_str()}', 
        step=0, 
        profiler_outdir=LOGDIR_PATH
    )
tf.summary.trace_off()

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


Or use `get_concrete_function` to extract the graph

In [21]:
cf_graph = combine_two_tensors.get_concrete_function(A, B).graph

for op in cf_graph.get_operations():
    print(op)

name: "first_tensor"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "first_tensor"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}

name: "second_tensor"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "second_tensor"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}

name: "add"
op: "AddV2"
input: "first_tensor"
input: "second_tensor"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}

name: "Identity"
op: "Identity"
input: "add"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



## Encode tensors into serialized example_pb2

There is a function [tf.io.encode_proto](https://www.tensorflow.org/api_docs/python/tf/io/encode_proto) with very little information on how to use it. Will leave finding this out unitl a suitable cases arises. 